In [ ]:
import numpy as np
import matplotlib.pylab as plt

In [ ]:
n = 100
m = 200

A = np.random.rand(n,m)

In [ ]:
u, s, vh = np.linalg.svd(A, full_matrices=False)

Generate a random matrix and calcualte the SVD
We can see that the some singular values are large compared to outers.
We want to extract these alrge singular values with the matching rank-matricies $s_i u_i v_i^\top $
In this case we have $s = 0$

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

axs[0].scatter(s,np.arange(1,s.size+1))
axs[1].hist(s,bins = 100);


In [ ]:
max(s)

In [ ]:
plt.hist(np.abs(u.flatten()),bins = 50);

In [ ]:
s[0]

In [ ]:
R=A-(s[0]*u[:,0].reshape(n,1))@(vh[0,:].reshape(1,m))
u_n, s_n,vh_n = np.linalg.svd(R,full_matrices=False)

Now we have the rest. Here we thershold the vectors by using $\gamma max(v_i)$ with $0 < \gamma < 1$.

Note: because all vecotrs have equal lenght this leads to the fact that more vlaues are set to 0 for a vector with a larger max which also means that the other values are smaller.

Note II: here we basically guess the next element out of a overcomplete dictionary. Eventually this could be done in a better way (some best approximation of Matrix in Rank1 wiht 1-norm that promotes sparsity...) 
but here we hope that by chance one element seperates form the rest

In [ ]:
max_u = np.max(np.abs(u_n),axis=0)
max_u.shape=(1,s_n.size)

max_v = np.max(np.abs(vh_n),axis=1)
max_v.shape=(s_n.size,1)
u_n[np.abs(u_n)<0.4*max_u]=0
vh_n-max_v
vh_n[np.abs(vh_n)<0.4*max_v]=0

Normalize thm again

In [ ]:
u_n=u_n/np.linalg.norm(u_n,axis=0)
vh_n=vh_n/np.linalg.norm(vh_n,axis=1).reshape(s_n.size,1) 

In [ ]:
plt.hist(abs(u_n.flatten()))

In [ ]:
for i in range(0,s_n.size):
    s_n[i] = np.sum(R*(u_n[:,i].reshape(n,1)@(vh_n[i,:].reshape(1,m))))

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

axs[0].scatter(abs(s_n),np.arange(1,s.size+1))
axs[1].hist(abs(s_n),bins = 100);


In [ ]:
k = np.argmax(s_n)
display(k)
display(s_n[k])

In [ ]:
def approx(Us,a,Vs):
    #calcaultes the approxiamtion
    
    #Us: Matrix, collums are us
    #a:  factors
    #Vs: Matrix, collumns are vs
    return Us@np.diag(a)@Vs.T

In [ ]:
u, s, vh = np.linalg.svd(A, full_matrices=False)
np.max(abs(A-approx(u,s,vh.T)))

In [ ]:
def inner(A,u,v):
    #calcaulates the inner product of flatten(A) with flatten(u@v.T)
    n = u.size
    m = v.size
    return np.sum(A*(u.reshape(n,1)@(v.reshape(1,m))))

In [ ]:
display(inner(np.eye(3),np.array([0,1,0]),np.array([0,1,0])))
display(inner(np.eye(3),np.array([1,0,0]),np.array([0,1,0])))

In [ ]:
def sparsify(Q):
    #Takes Matrix Q with collumn vectors and makes them sparse
    max_q = np.max(np.abs(Q),axis=0)
    max_q.shape=(1,Q.shape[1])


    Q[np.abs(Q)<gamma*max_q]=0
    #renormalize them
    Q=Q/np.linalg.norm(Q,axis=0)
    return Q

In [ ]:
gamma = 0.2
Q = np.eye(4)
Q[1,2]=0.5
Q[1,3]=1
Q[3,0]=0.1
display(Q)
Q = sparsify(Q)
display(Q)
display(np.diag(Q.T@Q))

In [ ]:
def matrixnorm(u,v):
    n = u.size
    m = v.size
    return np.linalg.norm((u.reshape(n,1)@(v.reshape(1,m))))
    

In [ ]:
def calc_new_atom(R,plot_hist = False):
    #R is the current residuum
    #calcualte the svd
    U, s, Vh = np.linalg.svd(R, full_matrices=False)

    #sparsify them
    U = sparsify(U)
    V = sparsify(Vh.T)
    
    for i in range(0,s.size):
        U[:,i] = U[:,i]/matrixnorm(U[:,i],V[:,i])
        s[i] = inner(R,U[:,i],V[:,i]) 
#we maybee would need a sign correction here but ignore it for now it is unlikeley that a vector would become so negative
    
    if plot_hist:
        axs[0].scatter(s,np.arange(0,s.size))
        axs[1].hist(s,bins = 100);
    k = np.argmax(s)
    u = U[:,k]
    v = V[:,k]
    return u,s[k],v

In [ ]:
A = np.array([[0,-1,0,0,0,1,1]]).T@np.array([[1,0,0,0,0,-1,1]])
display(A)
u,s,v = calc_new_atom(A,plot_hist = False)
display(s*u.reshape(7,1)@v.reshape(1,7))
display(approx(u.reshape(7,1),np.array([s]),v.reshape(7,1)))

In [ ]:
gamma = 0.7
i = 0 #index for Array
n = 200
m = 200
A = np.random.standard_normal((n,m))
d = 700 #number of matricies
Us = np.zeros((n,d))
Vs = np.zeros((m,d))
a = np.zeros(d)

In [ ]:
for i in range(d):
    gamma = 0.3*(d-i)/d+0.5*(i)/d
    A_tilde = approx(Us,a,Vs)
    R = A-A_tilde
    print('||R||=',np.linalg.norm(R),'gamma=',gamma)
    u,s,v = calc_new_atom(R,plot_hist = False)
    Us[:,i]=u
    Vs[:,i]=v
    a[i]=s


In [ ]:
def number_of_params(Qs):
    #returns a vector with the number of nonzero eleemts for each vector
    return np.count_nonzero(Qs,axis = 0)

In [ ]:
plt.plot(number_of_params(Us),'x')
plt.plot(number_of_params(Vs),'x')

In [ ]:
np.count_nonzero(Us)

In [ ]:
np.count_nonzero(Vs)

In [ ]:
m*n